***
# Détermination des sujets des posts
***

In [7]:
# ====== Importer les library ====== #
import pandas as pd
import numpy as np
import concurrent.futures
from transformers import pipeline

pd.set_option('display.max_columns', None)

In [2]:
# ====== Déterminer les path ====== #
import os
cwd = os.getcwd()
parent = os.path.dirname(cwd)

data_path = parent + '\\data\\'
df_path = cwd + '\\dataframeTest\\'

In [3]:
posts = pd.read_csv(data_path + 'Posts.csv')

In [4]:
len(posts)

37504

In [9]:
def modiliserSujetDesArticles(titre_list, batch_size=10):
    classifier = pipeline(model="facebook/bart-large-mnli")
    result_list = []

    candidate_labels=[
    "Environnement",
    "Politique",
    "COVID",
    "Technologie",
    "Économie",
    "Justice",
    "Éducation",
    "Santé",
    "Sport"
    ]


    # Process texts in batches
    with concurrent.futures.ThreadPoolExecutor() as executor:

        for i in range(0, len(titre_list), batch_size):
            batch = titre_list[i:i + batch_size]
            results = executor.map(classifier, batch, [candidate_labels]*len(batch))
            for res in results:
                try: 
                    final_result = max(zip(res['scores'], res['labels']))[1]
                    result_list.append(final_result)
                except Exception as e:
                    print('KeyError:', e)
                    print('result :', res )
                    pass

    return result_list

In [12]:
print(len(posts))
df = posts[['title']].drop_duplicates().dropna().reset_index(drop=True).copy()
print(len(df))

37504
36015


In [13]:
titre_list = df['title'].tolist()
result_list = modiliserSujetDesArticles(titre_list)
df['title_classify'] = result_list

#568 

In [ ]:
df.to_hdf(df_path + 'df_topics.h5', key='s')